In [1]:
# !pip install -Uq transformers
# !pip install -Uq  langchain langchain-community langchain-experimental
# !pip install -Uq 'crewai[tools]'
# !pip install -Uq langchain_groq

In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Setting Up Sample SQL Database from Excel File

In [3]:
from sqlalchemy import create_engine

In [4]:
path = "/content/master_db.db"
engine = create_engine(f"sqlite:///{path}")

In [5]:
import pandas as pd
df = pd.read_csv('/content/titanic.csv')

In [6]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [7]:
df.to_sql('titanic', engine, index=False, if_exists='replace')

891

In [8]:
from langchain_community.utilities.sql_database import SQLDatabase

In [9]:
db = SQLDatabase.from_uri(f"sqlite:///{path}")

In [10]:
db.get_usable_table_names()

['titanic']

## Set Up OpenAI gpt-4o-mini LLM

In [11]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")

Enter your OPENAI_API_KEY: ··········


In [12]:
import os

os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [13]:
from crewai import LLM

llm = LLM(
    model="openai/gpt-4o-mini",
    temperature=0
)

In [14]:
llm.call("Hi, How are you today?")

"Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

## Setting up tools from Langchain

In [15]:
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLDataBaseTool,
    QuerySQLCheckerTool
)
from crewai.tools import tool

In [16]:
@tool("list_tables")
def list_tables() -> str:
  """List the available tables in the database"""
  return ListSQLDatabaseTool(db=db).invoke("")

In [17]:
list_tables.run()

Using Tool: list_tables


'titanic'

In [18]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
  """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
  """

  return InfoSQLDatabaseTool(db=db).invoke(tables)

In [19]:
print(tables_schema.run("titanic"))

Using Tool: tables_schema

CREATE TABLE titanic (
	"PassengerId" BIGINT, 
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"SibSp" BIGINT, 
	"Parch" BIGINT, 
	"Ticket" TEXT, 
	"Fare" FLOAT, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/


In [20]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

In [21]:
execute_sql.run("SELECT * FROM titanic WHERE age > 1 LIMIT 5")

Using Tool: execute_sql


<ipython-input-20-fe9d3b7f4430>:4: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  return QuerySQLDataBaseTool(db=db).invoke(sql_query)


"[(1, 0, 3, 'Braund, Mr. Owen Harris', 'male', 22.0, 1, 0, 'A/5 21171', 7.25, None, 'S'), (2, 1, 1, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'female', 38.0, 1, 0, 'PC 17599', 71.2833, 'C85', 'C'), (3, 1, 3, 'Heikkinen, Miss. Laina', 'female', 26.0, 0, 0, 'STON/O2. 3101282', 7.925, None, 'S'), (4, 1, 1, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'female', 35.0, 1, 0, '113803', 53.1, 'C123', 'S'), (5, 0, 3, 'Allen, Mr. William Henry', 'male', 35.0, 0, 0, '373450', 8.05, None, 'S')]"

## Loading Tasks and Agents YAML files

In [22]:
import yaml

files = {
    'agents': '/content/agents.yaml',
    'tasks': '/content/tasks.yaml'
}

configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

agents_config = configs['agents']
tasks_config = configs['tasks']

## Setting up Agents from CrewAI

In [23]:
from crewai import Agent, Crew, Process, Task, LLM

In [24]:
sql_dev = Agent(
    config=agents_config['sql_dev'],
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql],
    allow_delegation=False
)

In [25]:
data_analyst = Agent(
    config=agents_config['data_analyst'],
    llm=llm,
    allow_delegation=False,
)

In [26]:
report_writer = Agent(
    config=agents_config['report_writer'],
    llm=llm,
    allow_delegation=False,
)

## Setting up Tasks using CrewAI

In [27]:
extract_data = Task(
    config=tasks_config['extract_data'],
    agent=sql_dev,
)

In [28]:
analyze_data = Task(
    config=tasks_config['analyze_data'],
    agent=data_analyst,
    context=[extract_data],
)

In [29]:
write_report = Task(
    config=tasks_config['write_report'],
    agent=report_writer,
    context=[analyze_data],
)

## Setting up Crew

In [30]:
crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=True,
    memory=False,
    output_log_file="crew.log",
)

In [31]:
inputs = {
    "query": "Analyze the relationship between age and survival chance"
}

result = crew.kickoff(inputs=inputs)

# Agent: Senior Database Developer
## Task: Extract data that is required for the query Analyze the relationship between age and survival chance.



# Agent: Senior Database Developer
## Thought: I need to first identify the available tables in the database to find relevant data related to age and survival chance.
## Using tool: list_tables
## Tool Input: 
"{}"
## Tool Output: 
titanic


# Agent: Senior Database Developer
## Thought: Thought: The available table is "titanic". I will now check the schema of the "titanic" table to understand its structure and identify the relevant columns for age and survival chance.
## Using tool: tables_schema
## Tool Input: 
"{\"tables\": \"titanic\"}"
## Tool Output: 

CREATE TABLE titanic (
	"PassengerId" BIGINT, 
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"SibSp" BIGINT, 
	"Parch" BIGINT, 
	"Ticket" TEXT, 
	"Fare" FLOAT, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from titanic table:
PassengerId	Survived

In [32]:
print(result)

### Executive Summary

- **Objective**: Analyze the relationship between age and survival chance using a dataset of 100 entries.
- **Data Overview**: Ages range from 0 to 80 years; survival chances vary significantly.
  
#### Key Observations:
1. **Infants (0-5 years)**: High variability; critical ages show lower survival rates.
2. **Children (6-12 years)**: Generally high survival rates; ages 12 and 13 have a survival chance of 1.0.
3. **Adolescents (13-19 years)**: Variable survival rates; critical ages show increased risks.
4. **Young Adults (20-30 years)**: Mixed survival chances; early twenties face higher risks.
5. **Middle-Aged Adults (30-50 years)**: Stable survival rates around 0.4 to 0.6.
6. **Older Adults (50+ years)**: Declining survival rates, especially in late sixties and seventies.

- **Conclusion**: Age significantly impacts survival outcomes; targeted interventions are needed for vulnerable groups. Further statistical analysis is recommended for deeper insights.
